Define the growth function which takes in the current state of the system y, the parameters p, and the presence or absence of inhibitors inh as arguments. This function describes the rates of change of nutrients n, the degrading enzyme b, and antibiotic a levels over time t under specific conditions. 

The simulate function is used to solve the defined ODEs for a given initial condition n0 and a0 over a certain time span using the solve_ivp function from the SciPy library. This function returns the simulated levels of nutrients, the degrading enzyme, and antibiotic over time.

The objective function computes the sum of squared differences between the simulated nutrient levels and the observed experimental nutrient levels for all time points and initial conditions. This function is typically used in parameter estimation where the goal is to find the parameters p that minimize the difference between the simulated and experimental results.

Here, the growth function is defined to be flexible enough to capture the general trend of experimental growth curves. Many of these exhibit approximately 2-stage growth. A simple logistic model or Monod equation does not capture the growth curve well. The term 'g' is re-defined to approximate this behavior (to some extent). There might be better alternatives.

In the following code, the initial parameter values are from the NN estimated. Let's see if Sciy optimize can generate better fits for some or all isolates.

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

%run ../simulation/growth.ipynb   #defines growth dynamics and simulation method
#
# 1st round of parameter estimation.
#
# Bounds for the parameter values
# mumax, Ks, theta, Ln, kappab, phimax, gamma, betamin, db, c
parameter_names = ['mumax','Ks','theta', 'Ln', 'kappab', 'phimax', 'gamma', 'betamin', 'db', 'c']

# load the experimental data
experimental_all = np.load('../isolates/avg_OD_all_isolates.npy')
strain_ids = np.load('../isolates/strain_ids.npy', allow_pickle=True)
#old = np.load('../simulated_v3/NN_estimated/growth_rates.npy')
#old = np.mean(old, axis = 1)
print("experimental_all shape:", experimental_all.shape)
num_parameter_sets = 311 # Run the optimization for each strain, using averaged growth curves.
num_runs = 10

for k in range(num_runs):
    optimal_parameters = []
    initial_guesses = []
    for i in range(num_parameter_sets):
        experimental = experimental_all[i]

        p0 = np.ones(10)
        #these parameter bounds are manually selected based on the rough adjustment of base parameter values to generate a reasonable growth curve, as well as values from a previous model.
        #when estimating growth parameters, keep all other parameters constant. They don't contribute to optimization when using
        # 'objective_drug_free'. 
        parameter_bounds = [(0, 2), (0, 0.4), (0, 5), (1, 1), (1, 1), (1, 1),
                            (1, 1), (1, 1), (1, 1), (1, 1)]
        #set the means and standard deviations of the normal distribution
        parameter_means = [1, 0.2, 2.5, 1, 1, 1, 1, 1, 1, 1]
        parameter_sds = [1/3, 0.2/3, 2.5/3, 0, 0, 0, 0, 0, 0, 0]

        for j in range(0, 3):
            p0[j] = np.random.normal(parameter_means[j], parameter_sds[j])
        print(p0)  
        initial_guesses.append(p0)
    
        # Define the constraint function for parameter bounds, which are required by other optimization methods.
        #constraint = {'type': 'ineq', 'fun': lambda p: np.array([p[j] - parameter_bounds[j][0] for j in range(len(p))])}[]
        #p0[0:3] = old[i, 0:3]
        result = minimize(objective_drug_free, p0, args=(experimental,), method='Nelder-Mead', tol = 1e-6, bounds=parameter_bounds)
        optimal_parameters.append(result.x)
        interval = 1 if i<10 else 10
        if ((i+1) % interval == 0):
            print(strain_ids[i])
            #for p in optimal_parameters[i]:
            for j in range(len(optimal_parameters[i])):
                name = parameter_names[j]
                p = optimal_parameters[i][j]
                print(name, f"={p:.2f}", end="; ")
            print()  # Print a new line after each simulation

    # Convert the list of optimized parameters to a numpy array
    initial_guess_array = np.array(initial_guesses)
    optimal_parameters_array = np.array(optimal_parameters)
    # Save the optimized parameters to a .npy file
    np.save(('../estimated/scipy_growthrates_v2_' + str(k)), optimal_parameters_array)
    np.save(('../estimated/scipy_growthrateinitials_v2_' + str(k)), initial_guess_array)


experimental_all shape: (311, 3, 145)
[1.05309581 0.1775139  3.49732912 1.         1.         1.
 1.         1.         1.         1.        ]
2004
mumax =0.87; Ks =0.11; theta =1.83; Ln =1.00; kappab =1.00; phimax =1.00; gamma =1.00; betamin =1.00; db =1.00; c =1.00; 
[0.72777555 0.15572202 1.09036779 1.         1.         1.
 1.         1.         1.         1.        ]
2005
mumax =1.09; Ks =0.12; theta =1.90; Ln =1.00; kappab =1.00; phimax =1.00; gamma =1.00; betamin =1.00; db =1.00; c =1.00; 
[1.11875742 0.21420848 1.57596157 1.         1.         1.
 1.         1.         1.         1.        ]
2007
mumax =0.88; Ks =0.17; theta =2.63; Ln =1.00; kappab =1.00; phimax =1.00; gamma =1.00; betamin =1.00; db =1.00; c =1.00; 
[0.5503169  0.15695218 2.80405458 1.         1.         1.
 1.         1.         1.         1.        ]
2009
mumax =0.84; Ks =0.19; theta =3.64; Ln =1.00; kappab =1.00; phimax =1.00; gamma =1.00; betamin =1.00; db =1.00; c =1.00; 
[0.76985249 0.22773743 2.62354843 

/tmp/ipykernel_927926/3045610688.py:46: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(objective_drug_free, p0, args=(experimental,), method='Nelder-Mead', tol = 1e-6, bounds=parameter_bounds)


[1.6874879  0.20552411 1.81058624 1.         1.         1.
 1.         1.         1.         1.        ]
[1.23751438 0.2659343  2.5894459  1.         1.         1.
 1.         1.         1.         1.        ]
[0.47100616 0.25934685 2.8252618  1.         1.         1.
 1.         1.         1.         1.        ]
[0.80957522 0.19738927 3.44398746 1.         1.         1.
 1.         1.         1.         1.        ]
[0.59582614 0.17160543 1.72763302 1.         1.         1.
 1.         1.         1.         1.        ]
[1.01109102 0.21984883 2.13628073 1.         1.         1.
 1.         1.         1.         1.        ]
[0.7631116  0.23214423 2.46063402 1.         1.         1.
 1.         1.         1.         1.        ]
[1.28980435 0.18636704 2.25635905 1.         1.         1.
 1.         1.         1.         1.        ]
2175
mumax =0.79; Ks =0.17; theta =3.25; Ln =1.00; kappab =1.00; phimax =1.00; gamma =1.00; betamin =1.00; db =1.00; c =1.00; 
[0.82692983 0.22893918 3.48704357

The following code checks the quality of estimated parameters
1. Load the estimated parameters 
2. For each parameter set, conduct simulations with the defined timespan for the three conditions: (0, 0), (10, 0), (3, 10)
3. In each simulation, n0 is set by using the first value of the corresponding experimental data
4. For each parameter set, generate a 3-panel plot. Each panel shows the overlay of simulated data and experimental data for one of the 3 conditions.
5. When doing the optimization, the experimental time points are not considered. Instead, a reasonable time span is used to simulate time courses. It is close but not identical to experimental time span. This can be corrected later by re-scaling the time axis of the simulation. 